<a href="https://colab.research.google.com/github/HrithikVaya007/Churn-Predictor-App/blob/main/churn_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
code = """

import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

st.set_page_config(layout="wide")
st.title("📉 Universal Customer Churn Predictor")
st.markdown("Upload your **customer churn dataset** (.csv) and predict churn probability using **Polynomial Regression**.")

# File upload
st.subheader("📂 Step 1: Upload Your Dataset")
uploaded_file = st.file_uploader("Choose a CSV file", type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file)


    for col in ['TotalCharges', 'MonthlyCharges']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Preview data
    with st.expander(" Dataset Preview (first 5 rows)"):
        st.dataframe(df.head())

    # Detect churn column
    binary_candidates = [col for col in df.columns if df[col].nunique() == 2]
    churn_name_matches = [col for col in df.columns if any(key in col.lower() for key in ['churn', 'exited', 'target'])]
    default_churn = churn_name_matches[0] if churn_name_matches else (binary_candidates[0] if binary_candidates else None)

    if not default_churn:
        st.error(" Could not detect a binary churn column. Ensure your dataset has a 0/1 or Yes/No target column.")
        st.stop()

    st.subheader(" Step 2: Choose Columns and Parameters")
    col1, col2 = st.columns(2)

    with col1:
        churn_column = st.selectbox(" Churn Column (Binary)", binary_candidates, index=binary_candidates.index(default_churn))

    y_raw = df[churn_column]
    if y_raw.dtype == 'O':
        y = y_raw.astype(str).str.strip().str.lower().map({'yes': 1, 'no': 0, '1': 1, '0': 0})
    else:
        y = y_raw.copy()

    if y.isna().all():
        st.error(" Churn column could not be interpreted as binary (Yes/No or 0/1).")
        st.stop()

    numeric_columns = df.select_dtypes(include=[np.number]).columns
    feature_candidates = [col for col in numeric_columns if col.lower() not in ['seniorcitizen', 'customerid'] and col != churn_column]

    if not feature_candidates:
        st.error(" No valid numeric feature columns found for prediction.")
        st.stop()

    with col2:
        default_feature = 'MonthlyCharges' if 'MonthlyCharges' in feature_candidates else feature_candidates[0]
        feature_column = st.selectbox("Numeric Feature for Prediction", feature_candidates, index=feature_candidates.index(default_feature))

    degree = st.slider(" Polynomial Degree", 1, 5, 2)

    # Clean rows
    mask = (~y.isna()) & (~df[feature_column].isna())
    if mask.sum() == 0:
        st.error("⚠ No valid rows found after cleaning.")
        st.stop()

    X = df.loc[mask, [feature_column]]
    y_clean = y[mask]

    # Model training
    st.subheader(" Step 3: Train & Evaluate Model")
    with st.spinner(" Training model..."):
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)
        model = LinearRegression()
        model.fit(X_poly, y_clean)
        y_pred = model.predict(X_poly)

    mse = mean_squared_error(y_clean, y_pred)
    st.success(f" Model Trained! Mean Squared Error (MSE): `{mse:.4f}`")
    st.metric(label=" Model Error (MSE)", value=f"{mse:.4f}")

    # Show predictions
    df_result = df.loc[mask].copy()
    df_result['Predicted Churn Probability'] = y_pred
    with st.expander(" Prediction Results Table"):
        st.dataframe(df_result[[feature_column, churn_column, 'Predicted Churn Probability']])

    # Plot
    st.subheader(" Step 4: Visualize Predictions")
    plt.figure(figsize=(10, 5))
    plt.scatter(X, y_clean, color='skyblue', label='Actual', alpha=0.6)
    plt.plot(X, y_pred, color='red', label='Predicted', linewidth=2)
    plt.xlabel(feature_column)
    plt.ylabel("Churn Probability")
    plt.title("Actual vs Predicted Churn")
    plt.legend()
    st.pyplot(plt)

    # Footer
    st.markdown("---")
    st.markdown(" Built with  using Streamlit | [GitHub](https://github.com)")


"""



In [23]:

with open("app.py", "w") as f:
    f.write(code)

!pkill -f streamlit
!pkill -f ngrok
PORT = 7860
!streamlit run app.py --server.port {PORT} &>/dev/null &
from pyngrok import ngrok
public_url = ngrok.connect(PORT)
print(f" Your app is live at: {public_url}")

 Your app is live at: NgrokTunnel: "https://428497ea1b91.ngrok-free.app" -> "http://localhost:7860"


In [24]:
from pyngrok import conf
conf.get_default().auth_token = "31ByqE1BLmLk5lXgnRB3VhngGGK_69efqj9DVjLvNdoyxCZ7j"



In [25]:
pip install streamlit pandas numpy scikit-learn matplotlib ngrok
